In [2]:
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('vectorized_data.zip')

In [4]:
df.head()

,Polls ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score,time_stamp,tokenized_responses,stemmed_responses,clean_responses,string,LOs/ HCs,...,vec_11,vec_12,vec_13,vec_14,vec_15,vec_16,vec_17,vec_18,vec_19,vec_20
0,12522,The strengths of Plato's approach is his const...,#objectivemorality,2,1,"['The', 'strengths', 'of', ""Plato's"", 'approac...","['the', 'strength', 'of', 'plato', 'approach',...","['strength', 'plato', 'approach', 'construct',...",strength plato approach construct whole framew...,objmorality,...,0.227378,0.228062,-0.065646,-0.018259,0.589726,0.165162,0.243406,0.041697,0.047487,-0.080301
1,12522,In the breakout we discussed if outside the ca...,#objectivemorality,3,1,"['In', 'the', 'breakout', 'we', 'discussed', '...","['in', 'the', 'breakout', 'we', 'discuss', 'if...","['breakout', 'discuss', 'outsid', 'cave', 'big...",breakout discuss outsid cave bigger cave thus ...,objmorality,...,0.067890,-0.172406,-0.026694,-0.313533,0.335412,-0.184949,-0.000822,0.093128,0.022665,0.142223
2,12522,Back to cmmon confusion time: the section 'und...,#objectivemorality,2,1,"['Back', 'to', 'cmmon', 'confusion', 'time', '...","['back', 'to', 'cmmon', 'confus', 'time', 'the...","['back', 'cmmon', 'confus', 'time', 'section',...",back cmmon confus time section understand inte...,objmorality,...,0.172736,0.221275,0.215809,-0.060970,0.372512,-0.060175,0.251572,0.366057,0.229787,0.318017
3,12522,Most difficult weakness is that his position w...,#objectivemorality,2,1,"['Most', 'difficult', 'weakness', 'is', 'that'...","['most', 'difficult', 'weak', 'is', 'that', 'h...","['difficult', 'weak', 'posit', 'understand', '...",difficult weak posit understand testabl like i...,objmorality,...,0.134589,-0.047511,-0.054336,-0.222005,0.227974,-0.272034,0.009888,-0.173518,0.228723,0.078310
4,12522,I'm still trying to understand the significanc...,#objectivemorality,2,1,"[""I'm"", 'still', 'trying', 'to', 'understand',...","[""i'm"", 'still', 'tri', 'to', 'understand', 't...","[""i'm"", 'still', 'tri', 'understand', 'signifi...",i'm still tri understand signific cave analog ...,objmorality,...,0.025134,0.092645,-0.150749,-0.375577,-0.011666,-0.353730,0.131935,-0.431555,0.036446,-0.056576


In [5]:
X = df.drop('Assessment reports Score', axis=1)
y = df['Assessment reports Score']

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [6]:
import random

random.seed(42)

train_sample_indices = random.sample(list(X_train.index), k=1000)
test_sample_indices = random.sample(list(X_test.index), k=100)

small_X_train = X_train.loc[X_train.index.isin(train_sample_indices)]
small_X_test = X_test.loc[X_test.index.isin(test_sample_indices)]
small_y_train = y_train.loc[y_train.index.isin(train_sample_indices)]
small_y_test = y_test.loc[y_test.index.isin(test_sample_indices)]


In [7]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [46]:
X_train.columns

Index(['Polls ID', 'Poll Responses Response', 'Assessment reports Hashtag',
       'time_stamp', 'tokenized_responses', 'stemmed_responses',
       'clean_responses', 'string', 'LOs/ HCs', 'College', 'Course',
       'flesch_reading_ease', 'flesch_kincaid_grade', 'gunning_fog',
       'smog_index', 'automated_readability_index', 'coleman_liau_index',
       'dale_chall_readability_score', 'linsear_write_formula',
       'lex_data_prep', 'hdd', 'Summer', 'Dugast', 'words_count',
       'unique_words', 'ttr', 'rttr', 'cttr', 'mtld', 'herdan', 'maas', 'vec',
       'vec_1', 'vec_2', 'vec_3', 'vec_4', 'vec_5', 'vec_6', 'vec_7', 'vec_8',
       'vec_9', 'vec_10', 'vec_11', 'vec_12', 'vec_13', 'vec_14', 'vec_15',
       'vec_16', 'vec_17', 'vec_18', 'vec_19', 'vec_20'],
      dtype='object')

In [59]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification


# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize the text input and create input tensors
tokenized_texts = [tokenizer.tokenize(text) for text in df["Poll Responses Response"]]
input_ids = [tokenizer.convert_tokens_to_ids(text) for text in tokenized_texts]
input_tensors = tf.keras.preprocessing.sequence.pad_sequences(input_ids, padding='post')

# Create feature tensors
feature_tensors = df["dale_chall_readability_score"].values.reshape(-1, 1)

# Concatenate input and feature tensors
combined_tensors = tf.concat([input_tensors, feature_tensors], axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:

# Add batch dimension
batch_tensors = [tensor.expand_dims(0) for tensor in combined_tensors]

# Evaluate the sentiment of each input using BERT
# sentiments = []
# for batch_tensor in batch_tensors:
#     outputs = model(batch_tensor)
#     predictions = tf.argmax(outputs.logits, dim=1)
#     sentiments.append(predictions.item())

# # Add sentiment column to DataFrame
# df["sentiment"] = sentiments

# # Save the DataFrame with sentiment predictions
# df.to_csv("predictions.csv", index=False)


In [8]:
import torch

torch.has_mps
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def preprocess_function(examples):
   return tokenizer(examples["Poll Responses Response"], truncation=True)
 
tokenized_train = tokenizer(list(small_X_train), truncation=True, padding=True, return_tensors = 'tf', max_length=128)
tokenized_test = tokenizer(list(small_X_test), truncation=True, padding=True, return_tensors = 'tf', max_length=128)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-20 22:16:01.277988: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-20 22:16:01.278625: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
# from transformers import DataCollatorWithPadding
# import torch
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


# import numpy as np
# from datasets import load_metric
 
# def compute_metrics(eval_pred):
#    load_accuracy = load_metric("accuracy")
#    load_f1 = load_metric("f1")
  
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
#    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
#    return {"accuracy": accuracy, "f1": f1}


In [10]:
import transformers
from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [44]:
# Encode the training data
encoded_train_data = tokenizer(list(small_X_train['Poll Responses Response']), padding=True, truncation=True, return_tensors='tf')
encoded_test_data = tokenizer(list(small_X_test['Poll Responses Response']), padding=True, truncation=True, return_tensors='tf')

# Print the encoded data
print(encoded_train_data)

{'input_ids': <tf.Tensor: shape=(1000, 166), dtype=int32, numpy=
array([[ 101, 2057, 2288, ...,    0,    0,    0],
       [ 101, 2023, 2052, ...,    0,    0,    0],
       [ 101, 1044, 2064, ...,    0,    0,    0],
       ...,
       [ 101, 2013, 2000, ...,    0,    0,    0],
       [ 101, 2096, 1045, ...,    0,    0,    0],
       [ 101, 2057, 2064, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1000, 166), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}


In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import TFBertModel
import numpy as np 
from tqdm import tqdm

# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('distilbert-base-uncased')

# Define batch size
batch_size = 32

# Get the number of samples in the data
num_samples = len(encoded_train_data['input_ids'])

# Create a list to store the embeddings
embeddings = []

# Loop through the data in batches
for i in tqdm(range(0, num_samples, batch_size)):
    # Get a batch of data
    batch_input_ids = encoded_train_data['input_ids'][i:i+batch_size]
    batch_attention_mask = encoded_train_data['attention_mask'][i:i+batch_size]
    
    # Get the embeddings for the batch
    batch_outputs = bert_model(batch_input_ids, attention_mask=batch_attention_mask)
    
    # Add the embeddings to the list
    embeddings.extend(batch_outputs[0].numpy())

# Convert the embeddings list to a numpy array
embeddings_train = np.array(embeddings)


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'distilbert']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stre

In [13]:
embeddings_test = []
for i in tqdm(range(0, num_samples, batch_size)):
    # Get a batch of data
    batch_input_ids = encoded_test_data['input_ids'][i:i+batch_size]
    batch_attention_mask = encoded_test_data['attention_mask'][i:i+batch_size]
    
    # Get the embeddings for the batch
    batch_outputs = bert_model(batch_input_ids, attention_mask=batch_attention_mask)
    
    # Add the embeddings to the list
    embeddings_test.extend(batch_outputs[0].numpy())

# Convert the embeddings list to a numpy array
embeddings_test = np.array(embeddings_test)


100%|██████████| 32/32 [00:04<00:00,  7.98it/s]


In [42]:
import tensorflow as tf
from transformers import TFBertModel
from tensorflow.keras.layers import GlobalAveragePooling1D

# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_seq_length = 166
# Create a Keras model that produces the BERT embeddings
input_ids = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32)
attention_mask = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32)
embedding_layer = bert_model(input_ids, attention_mask=attention_mask)[0]
pooling_layer = GlobalAveragePooling1D()(embedding_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(pooling_layer)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

model.summary()
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_33 (InputLayer)          [(None, 166)]        0           []                               
                                                                                                  
 input_34 (InputLayer)          [(None, 166)]        0           []                               
                                                                                                  
 tf_bert_model_7 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_33[0][0]',               
                                thPoolingAndCrossAt               'input_34[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 166,                                         

In [36]:
encoded_train_data['input_ids'].shape,  encoded_train_data['attention_mask'].shape

(TensorShape([1000, 166]), TensorShape([1000, 166]))

In [40]:
small_y_train = np.asarray(small_y_train).astype('float32').reshape((-1,1))

In [45]:
# Train the model
batch_size = 32
model.fit(
    x=[encoded_train_data['input_ids'], encoded_train_data['attention_mask']],
    y=small_y_train,
    batch_size=batch_size,
    epochs=5,
    validation_split=0.2
)

Epoch 1/5
 2/25 [=>............................] - ETA: 7:47 - loss: -3.4875 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [14]:
# import keras modules
from keras.models import Sequential
from tensorflow.keras.models import Model
from keras.layers import Dense, Dropout, Input

In [15]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(small_y_train)
y_test_one_hot = to_categorical(small_y_test)


In [16]:
embeddings_train.shape

(1000, 166, 768)

In [28]:


# define the model architecture
input_ids = tf.keras.Input(shape=(embeddings_train.shape[1], embeddings_train.shape[2]), dtype=tf.int32)
attention_mask = tf.keras.Input(shape=(embeddings_train.shape[1], embeddings_train.shape[2]), dtype=tf.int32)
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_model([input_ids, attention_mask])

# bert_output, _ = bert_model([input_ids, attention_mask])
# output = tf.keras.layers.Dense(5, activation='sigmoid')(bert_output)
# model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# # compile the model
# optimizer = tf.keras.optimizers.Adam(lr=2e-5)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()


ValueError: Exception encountered when calling layer "tf_bert_model_1" (type TFBertModel).

in user code:

    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1088, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py", line 1115, in call  *
        outputs = self.bert(
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000gn/T/__autograph_generated_file2tfwx03d.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(self),), dict(**ag__.ld(unpacked_inputs)), fscope)
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000g

In [26]:
import tensorflow as tf
from transformers import TFBertModel

# define the model architecture
input_ids = tf.keras.Input(shape=(embeddings_train.shape[1], embeddings_train.shape[2]), dtype=tf.int32)
attention_mask = tf.keras.Input(shape=(embeddings_train.shape[1], embeddings_train.shape[2]), dtype=tf.int32)
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_output = bert_model([input_ids, attention_mask])[1]
output = tf.keras.layers.Dense(5, activation='sigmoid')(bert_output)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# compile the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


ValueError: Exception encountered when calling layer "tf_bert_model_1" (type TFBertModel).

in user code:

    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1088, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py", line 1115, in call  *
        outputs = self.bert(
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000gn/T/__autograph_generated_file2tfwx03d.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(self),), dict(**ag__.ld(unpacked_inputs)), fscope)
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000g

In [25]:
# define the model architecture
input_ids = tf.keras.Input(shape=(embeddings_train.shape[1], embeddings_train.shape[2]), dtype=tf.int32)

# input_ids = tf.keras.Input(embeddings_train.shape[1], dtype=tf.int32)
attention_mask = tf.keras.Input(shape=(128, ), dtype=tf.int32)
bert_output = bert_model([input_ids, attention_mask])[1]
output = tf.keras.layers.Dense(5, activation='sigmoid')(bert_output)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# compile the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


ValueError: Exception encountered when calling layer "tf_bert_model" (type TFBertModel).

in user code:

    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1088, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/transformers/models/bert/modeling_tf_bert.py", line 1115, in call  *
        outputs = self.bert(
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000gn/T/__autograph_generated_file2tfwx03d.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(self),), dict(**ag__.ld(unpacked_inputs)), fscope)
    File "/var/folders/0h/xyv81g2n7sj6zr0c9cw30gkc0000gn/

In [18]:
history = model.fit(embeddings_train, y_train_one_hot, epochs=10, validation_data=(embeddings_test, y_test_one_hot))

2023-04-20 22:16:49.676590: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


ValueError: in user code:

    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/swimmingcircle/miniforge3/e

In [58]:
# Define the neural network architecture
num_classes = 5
inputs = Input(shape=(embeddings_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(embeddings_train, y_train_one_hot, epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(embeddings_test, y_test_one_hot, batch_size=32)

Epoch 1/10


2023-04-20 22:00:26.269041: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/swimmingcircle/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/swimmingcircle/miniforge3/e

In [ ]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# from transformers import TFBertModel

# # Load the pre-trained BERT model
# bert_model = TFBertModel.from_pretrained('distilbert-base-uncased')

# # Get the embeddings for the encoded data
# outputs = bert_model(encoded_train_data['input_ids'], attention_mask=encoded_train_data['attention_mask'])


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFBertModel: ['vocab_transform', 'vocab_projector', 'distilbert', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
bert_model.summary()